# 3. EDA: Analisis exploratorio de casos

In [1]:
# manipulacion de archivos
import os

# transformacion de datos
import numpy as np
import pandas as pd
from itertools import chain

# graficos compatible con dash
import plotly.express as px

### 3.1. ¿Cuál es la heterogeneidad tecnologica al interior de grupos?

En esta seccion, exploramos las caracteristicas tecnologicas de los dos grupos de interes con el fin de entender un poco sobre su contexto tecnologico. Consideramos que ganar claridad sobre la heterogeneidad tecnologica al interior de grupos nos ayudará a entender la distribucion de la volatilidad al interior de cada grupo. 

In [2]:
data_market = pd.read_pickle(os.path.join('1_data','data_market.pkl'))
data_market.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34078 entries, 2072 to 56249
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   coin_id        34078 non-null  object 
 1   prices         34078 non-null  float64
 2   year           34078 non-null  int32  
 3   month          34078 non-null  int32  
 4   day            34078 non-null  int32  
 5   market_caps    34078 non-null  float64
 6   total_volumes  34078 non-null  float64
 7   category_id    34078 non-null  object 
 8   categories     34078 non-null  object 
dtypes: float64(3), int32(3), object(3)
memory usage: 2.2+ MB


#### 3.1.1. Caracteristicas tecnologicas del grupo 'layer-1'

In [3]:
# nos concentramos unicamente en el grupo 'layer-1'
data_coins_info_layer1 = data_market.loc[
    data_market['category_id'].str.contains('layer-1')][['coin_id','categories']].drop_duplicates(subset=['coin_id'])

# calculamos la frecuencia de cada una de las etiquetas listadas
# cada etiqueta corresponde a una caracteristica tecnologica
categories_freq_layer1=pd.DataFrame(
    pd.Series(list(chain.from_iterable(data_coins_info_layer1['categories'].dropna())))
    .value_counts()).reset_index().rename(columns={'index':'category'})

# nuevamente, las caracteristicas no son excluyentes, asi que una criptomoneda puede evidenciar multiples caracteristicas
# por esta razon, expresamos la frecuencia de cada caracteristica como porcentaje del total de casos del grupo;
# naturalmente, todos los casos del grupo Layer 1 (L1) son Layer 1 (L1);
# dejamos esta fila como confirmacion de la consistencia en la conformacion de grupos
categories_freq_layer1['count_shr']=round((categories_freq_layer1['count']/data_coins_info_layer1['coin_id'].nunique())*100,2)

# resultados
print(categories_freq_layer1.head(20))
categories_freq_layer1_top=categories_freq_layer1.iloc[:10]

                      category  count  count_shr
0                 Layer 1 (L1)     11     100.00
1      Smart Contract Platform      8      72.73
2           Ethereum Ecosystem      4      36.36
3       Alleged SEC Securities      4      36.36
4               Cryptocurrency      3      27.27
5          BNB Chain Ecosystem      3      27.27
6            Harmony Ecosystem      2      18.18
7        Exchange-based Tokens      2      18.18
8   Centralized Exchange (CEX)      2      18.18
9          Moonriver Ecosystem      2      18.18
10            Cosmos Ecosystem      2      18.18
11              Infrastructure      1       9.09
12             Tezos Ecosystem      1       9.09
13          Algorand Ecosystem      1       9.09
14            Cronos Ecosystem      1       9.09
15           Polygon Ecosystem      1       9.09
16                    Protocol      1       9.09
17             Canto Ecosystem      1       9.09
18           Cardano Ecosystem      1       9.09
19         Avalanche

Con base en lo anterior, observamos que el 72% de nuestro subconjunto de casos estan conformadas por casos con caracteristicas de `Smart Contract Platform`; esta tecnologia ser refiere a criptomoneda que incoporan en su algoritmo las condiciones contractuales de la transaccion, haciendo posible su realizacion automatica en caso que una de las condiciones se cumpla. De igual manera, el 36% son casos de `Alleged SEC Securities`; esta categoria corresponde a criptomonedas que estan vinculadas al valor de la organizacion que las genera según un fallo de la U.S. Securities and Exchange Commission (SEC). Adicionalmente, por lo menos 4 casos pertenecen a un ecosistema de criptomonedas.

### 3.1.2. Caracteristicas tecnologicas del grupo 'stablecoins'

In [4]:
# nos concentramos unicamente en el grupo 'stablecoins'
data_coins_info_stable = data_market.loc[
    data_market['category_id'].str.contains('stablecoins')][['coin_id','categories']].drop_duplicates(subset=['coin_id'])

# calculamos la frecuencia de cada una de las etiquetas listadas
# cada etiqueta corresponde a una caracteristica tecnologica
categories_freq_stable=pd.DataFrame(
    pd.Series(list(chain.from_iterable(data_coins_info_stable['categories'].dropna())))
    .value_counts()).reset_index().rename(columns={'index':'category'})

# nuevamente, las caracteristicas no son excluyentes, asi que una criptomoneda puede evidenciar multiples caracteristicas
# por esta razon, expresamos la frecuencia de cada caracteristica como porcentaje del total de casos del grupo;
# naturalmente, todos los casos del grupo stablecoins son stablecoins;
# dejamos esta fila como confirmacion de la consistencia en la conformacion de grupos
categories_freq_stable['count_shr']=round((categories_freq_stable['count']/data_coins_info_stable['coin_id'].nunique())*100,2)

# resultados
print(categories_freq_stable.head(20))
categories_freq_stable_top=categories_freq_stable.iloc[:10]

                   category  count  count_shr
0               Stablecoins     10      100.0
1        Ethereum Ecosystem     10      100.0
2         Polygon Ecosystem      7       70.0
3            USD Stablecoin      7       70.0
4         Harmony Ecosystem      6       60.0
5          Fantom Ecosystem      5       50.0
6       BNB Chain Ecosystem      5       50.0
7       Avalanche Ecosystem      5       50.0
8          Cronos Ecosystem      4       40.0
9        Moonbeam Ecosystem      4       40.0
10       Arbitrum Ecosystem      4       40.0
11          Velas Ecosystem      4       40.0
12  Near Protocol Ecosystem      4       40.0
13      Moonriver Ecosystem      4       40.0
14   Gnosis Chain Ecosystem      4       40.0
15          Metis Ecosystem      3       30.0
16         Solana Ecosystem      3       30.0
17       Optimism Ecosystem      3       30.0
18          Canto Ecosystem      2       20.0
19        Finance / Banking      2       20.0


A diferencia del grupo anterior (`Layer 1 (L1)`), la tabla de arriba sugiere que la enorme mayoria de los casos pertenece a algun ecosistema te criptomonedas. Observamos que el 100% de nuestro grupo `stablecoins` puede integrarse con el ecosistema `Ethereum Ecosystem`, seguido por el 70% de los casos que hacen parte tanto de algunos de los ecosistemas `USD Stablecoin` o `Polygon Ecosystem`.

Como mencionamos anteriormente, la membresia a un ecosistema particular no es excluyente de su membresia a otro. Esto sugiere que los casos de nuestro grupo `stablecoins` parecen ser relativamente flexbibles en terminos tecnologicos; es decir, son criptomonedas con una flexbilidad tecnologica suficiente que pueden empleadas en una amplia variedad de ecosistemas. Esto tambien ayuda a entender por que estas son las `stablecoins` con mayor el volumen de intercambio.

## 3.2. Distribucion de la volatilidad por año

### 3.2.1. Volatilidad en precios corrientes

In [5]:
data_market_prcvoltlty_crrnt = pd.read_pickle(os.path.join('2_pipeline','data_market_prcvoltlty_crrnt.pkl'))
data_market_prcvoltlty_crrnt.to_csv(os.path.join("3_output","data_market_prcvoltlty_crrnt.csv"))
data_market_prcvoltlty_crrnt.info()
data_market_prcvoltlty_crrnt.head()

<class 'pandas.core.frame.DataFrame'>
Index: 105 entries, 10 to 64
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   category_id    105 non-null    object 
 1   coin_id        105 non-null    object 
 2   year           105 non-null    int32  
 3   sd_coin        105 non-null    float64
 4   sd_group_mean  105 non-null    float64
 5   vltlty_ratio   105 non-null    float64
 6   vltlty_rank    105 non-null    float64
dtypes: float64(4), int32(1), object(2)
memory usage: 6.2+ KB


,category_id,coin_id,year,sd_coin,sd_group_mean,vltlty_ratio,vltlty_rank
10,layer-1,bitcoin,2019,2655.712480,255.73451,10.384646,1.0
15,layer-1,bitcoin-cash,2019,96.242064,255.73451,0.376336,2.0
35,layer-1,ethereum,2019,50.775028,255.73451,0.198546,3.0
5,layer-1,binancecoin,2019,8.374184,255.73451,0.032746,4.0
25,layer-1,cosmos,2019,1.132393,255.73451,0.004428,5.0


In [6]:
# create a group using groupby
groups_all = data_market_prcvoltlty_crrnt.groupby(['category_id','year'])
  
# size of group to count observations
groups_all = groups_all.size()
  
# make a column name 
groups_all.reset_index(name='observaciones')

,category_id,year,observaciones
0,layer-1,2019,11
1,layer-1,2020,11
2,layer-1,2021,11
3,layer-1,2022,11
4,layer-1,2023,11
5,stablecoins,2019,10
6,stablecoins,2020,10
7,stablecoins,2021,10
8,stablecoins,2022,10
9,stablecoins,2023,10


In [7]:
# transformos los datos a su log natural para facilitar su visualizacion
data_market_prcvoltlty_crrnt.loc[:,'vltlty_ratio_log'] = np.log(data_market_prcvoltlty_crrnt['vltlty_ratio'])

# Boxplot
boxplot_crrnt = px.box(
    data_market_prcvoltlty_crrnt#.loc[
        #data_market_prcvoltlty_crrnt['category_id'].str.contains('layer-1')]
    ,
    x='year',
    y='vltlty_ratio_log',
    points='all',
    facet_col='category_id',
    facet_col_wrap=2,
    labels={
        'year': 'año',
        'vltlty_ratio_log': 'Volatilidad relativa (nlog)',
        'category_id': 'Categoria'},
    title='Distribucion de la volatilidad relativa, por año (precios corrientes, USD$)')
 
boxplot_crrnt.add_hline(
    y=np.log(1),
    line_width=2,
    line_dash='dash',
    line_color='red')
boxplot_crrnt.update_xaxes(type='category')
boxplot_crrnt.show()

### 3.2.2. Variacion en la volatilidad segun contexto local

In [8]:
data_market_prcvoltlty_constant = pd.read_pickle(os.path.join('2_pipeline','data_market_prcvoltlty_constant.pkl'))
data_market_prcvoltlty_constant.info()
data_market_prcvoltlty_constant.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   category_id           420 non-null    object 
 1   code_pais             420 non-null    object 
 2   coin_id               420 non-null    object 
 3   year                  420 non-null    int32  
 4   sd_pais_coin          420 non-null    float64
 5   sd_group_pais_mean    420 non-null    float64
 6   sd_group_pais_median  420 non-null    float64
dtypes: float64(3), int32(1), object(3)
memory usage: 21.5+ KB


,category_id,code_pais,coin_id,year,sd_pais_coin,sd_group_pais_mean,sd_group_pais_median
0,layer-1,ARG,algorand,2019,0.765119,475.884650,0.765119
1,layer-1,ARG,algorand,2020,0.193741,782.470174,1.204551
2,layer-1,ARG,algorand,2021,0.758258,1739.810852,2.542353
3,layer-1,ARG,algorand,2022,0.463004,1403.234830,1.455064
4,layer-1,ARG,binancecoin,2019,15.583135,475.884650,0.765119


In [9]:
# create a group using groupby
data_market_prcvoltlty_constant_sd_group_pais=data_market_prcvoltlty_constant[
    ['category_id','code_pais','sd_group_pais_mean','sd_group_pais_median']
    ].copy().drop_duplicates()
data_market_prcvoltlty_constant_sd_group_pais.to_csv(os.path.join("3_output","data_market_prcvoltlty_constant_sd_group_pais.csv"))
groups_cntry = data_market_prcvoltlty_constant_sd_group_pais.groupby(['category_id','code_pais'])
  
# size of group to count observations
groups_cntry = groups_cntry.size()
  
# make a column name 
groups_cntry.reset_index(name='observaciones')

,category_id,code_pais,observaciones
0,layer-1,ARG,4
1,layer-1,BRA,4
2,layer-1,COL,4
3,layer-1,MEX,4
4,layer-1,USA,4
5,stablecoins,ARG,4
6,stablecoins,BRA,4
7,stablecoins,COL,4
8,stablecoins,MEX,4
9,stablecoins,USA,4


In [10]:
# transformos los datos a su log natural para facilitar su visualizacion
data_market_prcvoltlty_constant_sd_group_pais.loc[:,'sd_group_pais_mean_log'] = np.log(
    data_market_prcvoltlty_constant_sd_group_pais['sd_group_pais_mean'])

# Boxplot
boxplot_constant = px.box(
    data_market_prcvoltlty_constant_sd_group_pais#.loc[
        #data_market_prcvoltlty_constant_sd_group_pais['category_id'].str.contains('layer-1')]
        ,
    x='code_pais',
    y='sd_group_pais_mean_log',
    points='all',
    facet_col='category_id',
    facet_col_wrap=2,
    labels={
        'code_pais': 'pais',
        'sd_group_pais_mean_log': 'Promedio anual de Desv. std (nlog)',
        'category_id': 'Categoria'},
    title='Distribucion del promedio anual de Desv. std, por pais (precios constantes, USD$2017)')
boxplot_constant.update_xaxes(type='category')
boxplot_constant.show()

In [11]:
# transformos los datos a su log natural para facilitar su visualizacion
data_market_prcvoltlty_constant_sd_group_pais.loc[:,'sd_group_pais_median_log'] = np.log(
    data_market_prcvoltlty_constant_sd_group_pais['sd_group_pais_median'])

# Boxplot
boxplot_constant = px.box(
    data_market_prcvoltlty_constant_sd_group_pais,
    x='code_pais',
    y='sd_group_pais_median_log',
    points='all',
    facet_col='category_id',
    facet_col_wrap=2,
    labels={
        'code_pais': 'pais',
        'sd_group_pais_median_log': 'Mediana anual de Desv. std (nlog)',
        'category_id': 'Categoria'},
    title='Distribucion de la mediana anual de Desv. std, por pais (precios constantes)')
boxplot_constant.update_xaxes(type='category')
boxplot_constant.show()

## 3.3. Las criptomonedas relativamente mas estables: Indice de estabilidad relativa

In [12]:
def rescalar(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

data_market_vltltyindex = data_market_prcvoltlty_crrnt.loc[:,
    ['category_id','coin_id','year','vltlty_rank']
].copy()
data_market_vltltyindex_res=data_market_vltltyindex.groupby(
    ['category_id','year'])['vltlty_rank'].apply(rescalar)
data_market_vltltyindex_res=data_market_vltltyindex_res.reset_index(
    level=['category_id','year']).rename(
        columns={'vltlty_rank':'vltlty_rank_rscld'}).drop(columns=['category_id','year'])
data_market_vltltyindex=data_market_vltltyindex.join(data_market_vltltyindex_res)
data_market_vltltyindex.head(3)

,category_id,coin_id,year,vltlty_rank,vltlty_rank_rscld
10,layer-1,bitcoin,2019,1.0,0.0
15,layer-1,bitcoin-cash,2019,2.0,0.1
35,layer-1,ethereum,2019,3.0,0.2


In [13]:
#index_factor=1/(data_market_vltltyindex.groupby('category_id')['year'].nunique()) # layer-1: 0.2, stablecoins, 0.2
index_factor=1/(data_market_vltltyindex['year'].nunique())
data_market_vltltyindex['relstblty_index']=data_market_vltltyindex['vltlty_rank_rscld']*index_factor
data_vltltyindex=data_market_vltltyindex.groupby(
    ['category_id','coin_id'])['relstblty_index'].sum().reset_index().sort_values(
        by=['category_id','relstblty_index'],ascending=False)
data_vltltyindex.to_csv(os.path.join("3_output","data_vltltyindex.csv"))
data_vltltyindex

,category_id,coin_id,relstblty_index
18,stablecoins,tether,0.911111
12,stablecoins,binance-usd,0.733333
20,stablecoins,usd-coin,0.688889
19,stablecoins,true-usd,0.644444
13,stablecoins,dai,0.622222
16,stablecoins,paxos-standard,0.577778
11,stablecoins,bilira,0.377778
14,stablecoins,gemini-dollar,0.333333
17,stablecoins,stasis-eurs,0.111111
15,stablecoins,pax-gold,0.000000


### 3.3.1. Stablecoinds: Caracteristicas tecnologicas de las mas estables

In [14]:
data_vltltyindex_stablecoins_high=data_vltltyindex.loc[
    (data_vltltyindex['category_id'].str.contains('stablecoins')) &
    (data_vltltyindex['relstblty_index'] > 0.6)
    ].merge(data_coins_info_stable)
data_vltltyindex_stablecoins_low=data_vltltyindex.loc[
    (data_vltltyindex['category_id'].str.contains('stablecoins')) &
    (data_vltltyindex['relstblty_index'] < 0.6)
    ].merge(data_coins_info_stable)

In [15]:
# calculamos la frecuencia de cada una de las etiquetas listadas
# cada etiqueta corresponde a una caracteristica tecnologica
categories_freq_stable_high=pd.DataFrame(
    pd.Series(list(chain.from_iterable(data_vltltyindex_stablecoins_high['categories'].dropna())))
    .value_counts()).reset_index().rename(columns={'index':'category'})

# nuevamente, las caracteristicas no son excluyentes, asi que una criptomoneda puede evidenciar multiples caracteristicas
# por esta razon, expresamos la frecuencia de cada caracteristica como porcentaje del total de casos del grupo;
# naturalmente, todos los casos del grupo stablecoins son stablecoins;
# dejamos esta fila como confirmacion de la consistencia en la conformacion de grupos
categories_freq_stable_high['count_shr']=round((categories_freq_stable_high['count']/data_vltltyindex_stablecoins_high['coin_id'].nunique())*100,2)

# resultados
categories_freq_stable_high.to_csv(os.path.join("3_output","categories_freq_stable_high.csv"))
categories_freq_stable_high.head(20)

,category,count,count_shr
0,Fantom Ecosystem,5,100.0
1,USD Stablecoin,5,100.0
2,Ethereum Ecosystem,5,100.0
3,Stablecoins,5,100.0
4,Polygon Ecosystem,5,100.0
5,Harmony Ecosystem,5,100.0
6,Moonbeam Ecosystem,4,80.0
7,Velas Ecosystem,4,80.0
8,Arbitrum Ecosystem,4,80.0
9,Near Protocol Ecosystem,4,80.0


In [16]:
# calculamos la frecuencia de cada una de las etiquetas listadas
# cada etiqueta corresponde a una caracteristica tecnologica
categories_freq_stable_low=pd.DataFrame(
    pd.Series(list(chain.from_iterable(data_vltltyindex_stablecoins_low['categories'].dropna())))
    .value_counts()).reset_index().rename(columns={'index':'category'})

# nuevamente, las caracteristicas no son excluyentes, asi que una criptomoneda puede evidenciar multiples caracteristicas
# por esta razon, expresamos la frecuencia de cada caracteristica como porcentaje del total de casos del grupo;
# naturalmente, todos los casos del grupo stablecoins son stablecoins;
# dejamos esta fila como confirmacion de la consistencia en la conformacion de grupos
categories_freq_stable_low['count_shr']=round((categories_freq_stable_low['count']/data_vltltyindex_stablecoins_low['coin_id'].nunique())*100,2)

# resultados
categories_freq_stable_low.to_csv(os.path.join("3_output","categories_freq_stable_low.csv"))
categories_freq_stable_low.head(20)

,category,count,count_shr
0,Stablecoins,5,100.0
1,Ethereum Ecosystem,5,100.0
2,USD Stablecoin,2,40.0
3,Finance / Banking,2,40.0
4,Polygon Ecosystem,2,40.0
5,BNB Chain Ecosystem,1,20.0
6,Solana Ecosystem,1,20.0
7,Avalanche Ecosystem,1,20.0
8,TRY Stablecoin,1,20.0
9,Cryptocurrency,1,20.0


### 3.3.2. Layer-1: Caracteristicas tecnologicas de las mas estables

In [17]:
data_vltltyindex_layer1_high=data_vltltyindex.loc[
    (data_vltltyindex['category_id'].str.contains('layer-1')) &
    (data_vltltyindex['relstblty_index'] > 0.6)
    ].merge(data_coins_info_layer1)
data_vltltyindex_layer1_low=data_vltltyindex.loc[
    (data_vltltyindex['category_id'].str.contains('layer-1')) &
    (data_vltltyindex['relstblty_index'] < 0.5)
    ].merge(data_coins_info_layer1)

In [18]:
# calculamos la frecuencia de cada una de las etiquetas listadas
# cada etiqueta corresponde a una caracteristica tecnologica
categories_freq_layer1_high=pd.DataFrame(
    pd.Series(list(chain.from_iterable(data_vltltyindex_layer1_high['categories'].dropna())))
    .value_counts()).reset_index().rename(columns={'index':'category'})

# nuevamente, las caracteristicas no son excluyentes, asi que una criptomoneda puede evidenciar multiples caracteristicas
# por esta razon, expresamos la frecuencia de cada caracteristica como porcentaje del total de casos del grupo;
# naturalmente, todos los casos del grupo stablecoins son stablecoins;
# dejamos esta fila como confirmacion de la consistencia en la conformacion de grupos
categories_freq_layer1_high['count_shr']=round((categories_freq_layer1_high['count']/data_vltltyindex_layer1_high['coin_id'].nunique())*100,2)

# resultados
categories_freq_layer1_high.to_csv(os.path.join("3_output","categories_freq_layer1_high.csv"))
categories_freq_layer1_high.head(20)

,category,count,count_shr
0,Layer 1 (L1),5,100.0
1,Smart Contract Platform,4,80.0
2,Harmony Ecosystem,2,40.0
3,Ethereum Ecosystem,2,40.0
4,Alleged SEC Securities,2,40.0
5,Protocol,1,20.0
6,Infrastructure,1,20.0
7,Cryptocurrency,1,20.0
8,Fantom Ecosystem,1,20.0
9,Moonriver Ecosystem,1,20.0


In [19]:
# calculamos la frecuencia de cada una de las etiquetas listadas
# cada etiqueta corresponde a una caracteristica tecnologica
categories_freq_layer1_low=pd.DataFrame(
    pd.Series(list(chain.from_iterable(data_vltltyindex_layer1_low['categories'].dropna())))
    .value_counts()).reset_index().rename(columns={'index':'category'})

# nuevamente, las caracteristicas no son excluyentes, asi que una criptomoneda puede evidenciar multiples caracteristicas
# por esta razon, expresamos la frecuencia de cada caracteristica como porcentaje del total de casos del grupo;
# naturalmente, todos los casos del grupo stablecoins son stablecoins;
# dejamos esta fila como confirmacion de la consistencia en la conformacion de grupos
categories_freq_layer1_low['count_shr']=round((categories_freq_layer1_low['count']/data_vltltyindex_layer1_low['coin_id'].nunique())*100,2)

# resultados
categories_freq_layer1_low.to_csv(os.path.join("3_output","categories_freq_layer1_low.csv"))
categories_freq_layer1_low.head(20)

,category,count,count_shr
0,Layer 1 (L1),5,100.0
1,Smart Contract Platform,3,60.0
2,Alleged SEC Securities,2,40.0
3,Cryptocurrency,2,40.0
4,BNB Chain Ecosystem,2,40.0
5,Ethereum Ecosystem,2,40.0
6,Cosmos Ecosystem,1,20.0
7,Canto Ecosystem,1,20.0
8,Exchange-based Tokens,1,20.0
9,Centralized Exchange (CEX),1,20.0
